In [2]:
import pandas as pd
from datetime import datetime, timedelta
import copy
import random
import time
import math

# Definição do Problema e Parâmetros
SETUP_TIME_COLOR = 15
SETUP_TIME_PIECE = 3
DAILY_CAPACITY_MINUTES = 18 * 60 + 35

# Geração de dados simulados (mock data) com novas variáveis
mock_data = []
cores = ['PRETO', 'AZUL', 'BRANCO', 'VERMELHO', 'AMARELO', 'VERDE', 'ROSA', 'CINZA', 'LARANJA', 'ROXO']
product_codes = [f'1000101000{i}.0' for i in range(23, 253)]

for i in range(len(product_codes) * 3):
    code = random.choice(product_codes)
    cor = random.choice(cores)
    qtd_planejada = random.randint(100, 1500)
    estoque = random.randint(0, 1000)
    pedidos = qtd_planejada
    dias_entrega = random.randint(1, 20)
    tempo_calculado = round(random.uniform(0.1, 1.5), 4)
    pecas_por_gancheira = random.randint(2, 8)
    estoque_gancheira = random.randint(50, 200)

    mock_data.append({
        'CODIGO_PRODUTO': code,
        'Tinta': cor,
        'Quantidade_Planejada': qtd_planejada,
        'Estoque': estoque,
        'Pedidos': pedidos,
        'Data_de_Entrega': datetime.now() + timedelta(days=dias_entrega),
        'Tempo_Calculado': tempo_calculado,
        'Pecas_por_Gancheira': pecas_por_gancheira,
        'ESTOQUE_GANCHEIRA': estoque_gancheira
    })

# Funções do Algoritmo 

def calculate_cost(sequence):
    total_setup_cost = 0
    if len(sequence) < 2:
        return 0
    for i in range(len(sequence) - 1):
        if sequence[i]['Tinta'] != sequence[i+1]['Tinta']:
            total_setup_cost += SETUP_TIME_COLOR
        if sequence[i]['CODIGO_PRODUTO'] != sequence[i+1]['CODIGO_PRODUTO']:
            total_setup_cost += SETUP_TIME_PIECE
    return total_setup_cost

def calculate_prioritization_score(item):
    saldo = item['Estoque'] - item['Pedidos']
    due_date = item['Data_de_Entrega']
    prioridade_urgente = 1 if saldo >= 0 else 0
    prioridade_data = due_date.timestamp()
    prioridade_saldo = abs(saldo) if saldo < 0 else 0
    return (prioridade_urgente, prioridade_data, -prioridade_saldo)

def daily_sequencing_and_scheduling(unscheduled_items):
    schedule = []
    day_number = 1
    remaining_items = copy.deepcopy(unscheduled_items)

    while remaining_items:
        remaining_items.sort(key=calculate_prioritization_score)
        items_for_today_prioritized = []
        time_used_today_minutes = 0
        items_to_remove_from_main_list = []
        last_color = None
        last_piece_code = None
        
        for item in remaining_items:
            gancheiras_necessarias = math.ceil(item['Quantidade_Planejada'] / item['Pecas_por_Gancheira'])
            if gancheiras_necessarias > item['ESTOQUE_GANCHEIRA']:
                continue

            item_production_time_minutes = item['Tempo_Calculado'] * 60
            setup_cost_to_add = 0
            
            if items_for_today_prioritized:
                if last_color != item['Tinta']:
                    setup_cost_to_add += SETUP_TIME_COLOR
                if last_piece_code != item['CODIGO_PRODUTO']:
                    setup_cost_to_add += SETUP_TIME_PIECE

            if time_used_today_minutes + item_production_time_minutes + setup_cost_to_add <= DAILY_CAPACITY_MINUTES:
                items_for_today_prioritized.append(item)
                time_used_today_minutes += item_production_time_minutes + setup_cost_to_add
                last_color = item['Tinta']
                last_piece_code = item['CODIGO_PRODUTO']
                items_to_remove_from_main_list.append(item)
        
        if not items_for_today_prioritized:
            break
            
        color_groups = {}
        for item in items_for_today_prioritized:
            color = item['Tinta']
            if color not in color_groups:
                color_groups[color] = []
            color_groups[color].append(item)
        
        sorted_colors = sorted(color_groups.keys(), key=lambda c: calculate_prioritization_score(color_groups[c][0]))
        initial_daily_sequence = []
        for color in sorted_colors:
            initial_daily_sequence.extend(color_groups[color])

        current_day = {'day': day_number, 'items': initial_daily_sequence}
        schedule.append(current_day)
        remaining_items = [item for item in remaining_items if item not in items_to_remove_from_main_list]
        day_number += 1
        
    return schedule

# Busca Tabu (Otimizador)
def tabu_search_optimizer(daily_sequence, tabu_tenure=7, max_iterations=100):
    current_solution = list(daily_sequence)
    best_solution = list(daily_sequence)
    best_cost = calculate_cost(best_solution)
    tabu_list = []

    for iteration in range(max_iterations):
        best_neighbor = None
        best_neighbor_cost = float('inf')
        best_move = None
        for i in range(len(current_solution)):
            for j in range(i + 1, len(current_solution)):
                neighbor = list(current_solution)
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                move = tuple(sorted((i, j)))
                neighbor_cost = calculate_cost(neighbor)
                if move not in tabu_list and neighbor_cost < best_neighbor_cost:
                    best_neighbor = neighbor
                    best_neighbor_cost = neighbor_cost
                    best_move = move
        if best_neighbor:
            current_solution = best_neighbor
            tabu_list.append(best_move)
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)
            if best_neighbor_cost < best_cost:
                best_solution = best_neighbor
                best_cost = best_neighbor_cost
    return best_solution

# FUNÇÃO DE EXPORTAÇÃO
def export_schedule_to_excel(schedule, filename="cronograma_otimizado.xlsx"):
    processed_data = []
    for day_data in schedule:
        day_number = day_data['day']
        for order_in_day, item in enumerate(day_data['items'], 1):
            
            saldo = item['Estoque'] - item['Pedidos']
            
            row = {
                'Dia': day_number,
                'Ordem_no_Dia': order_in_day,
                'Saldo': saldo,  
                **item
            }
            processed_data.append(row)
    
    if not processed_data:
        print("Nenhum dado para exportar.")
        return

    df = pd.DataFrame(processed_data)
    df['Data_de_Entrega'] = df['Data_de_Entrega'].dt.strftime('%Y-%m-%d')
    
    
    column_order = [
        'Dia', 'Ordem_no_Dia', 'CODIGO_PRODUTO', 'Tinta', 
        'Quantidade_Planejada', 'Estoque', 'Pedidos', 'Saldo', 
        'Data_de_Entrega', 'Tempo_Calculado', 'Pecas_por_Gancheira', 'ESTOQUE_GANCHEIRA'
    ]
    # Filtra para usar apenas as colunas que existem no dataframe, caso alguma não exista
    existing_columns = [col for col in column_order if col in df.columns]
    df = df[existing_columns]

    try:
        df.to_excel(filename, index=False, engine='openpyxl')
        print(f"\nCronograma exportado com sucesso para o arquivo: '{filename}'")
    except Exception as e:
        print(f"\nErro ao exportar para Excel: {e}")
        print("Por favor, verifique se a biblioteca 'openpyxl' está instalada (pip install openpyxl).")

# Execução do Algoritmo Completo
if __name__ == "__main__":
    start_time = time.time()
    initial_schedule = daily_sequencing_and_scheduling(mock_data)
    optimized_schedule = []
    
    print("Otimizando a sequência de cada dia com a Busca Tabu...")
    for day_data in initial_schedule:
        refined_daily_sequence = tabu_search_optimizer(day_data['items'])
        optimized_schedule.append({
            'day': day_data['day'],
            'items': refined_daily_sequence
        })
    print("Otimização concluída.")
    
    # 
    export_schedule_to_excel(optimized_schedule)

    end_time = time.time()
    print(f"\nTempo de execução total: {end_time - start_time:.4f} segundos")

Otimizando a sequência de cada dia com a Busca Tabu...
Otimização concluída.

Cronograma exportado com sucesso para o arquivo: 'cronograma_otimizado.xlsx'

Tempo de execução total: 2.6366 segundos
